In [ ]:
from bs4 import BeautifulSoup as bs
import os, re
from datetime import datetime, timedelta
import pandas as pd
from pycotacao import get_exchange_rates, CurrencyCodes
import xlrd
from openpyxl import Workbook, load_workbook

tabela_siscomex = pd.read_excel('drive/MyDrive/Colab Notebooks/espelho/nova tabela siscomex.xlsx')

arq = os.listdir('drive/MyDrive/Colab Notebooks/espelho/xml/')

from google.colab import drive
drive.mount('/content/drive')

In [203]:
def extracao(soup):
  """Função utilizada a fim de extrair todas as informações relevantes da declaração de importação, para que em breve seja inserida na fatura final."""
  dis = {}
  dts = []
  for prod in soup.find_all('informacaoComplementar'):
    info = re.split("\n", prod.text)
    dis['IV'] = info[4].split()[-1]
    dis['Ref. Cliente'] = str('PO '+info[6].split()[-1])
    dis['Conhecimento'] = str(info[3].split()[-1])
  for prod in soup.find_all('cargaPesoBruto'):
    dis['Peso Bruto'] = float(prod.text)/100000
  for prod in soup.find_all('cargaPesoLiquido'):
    dis['Peso Liquido'] = float(prod.text)/100000
  for prod in soup.find_all('quantidadeVolume'):
    dis['Volume'] = (prod.text)
  for prod in soup.find_all('dataRegistro'):
    dts.append(str(prod.text))
    data_bruta = dts[0]
    arrumar_data = data_bruta[0:4] + '-' + data_bruta[4:6] + '-' + data_bruta[6:]
    data = datetime.strptime(arrumar_data, "%Y-%m-%d")
    dis['Data Registro'] = data
  for prod in soup.find_all('numeroDI'):
    dis['Numero DI'] = (prod.text[:2] + '/' + prod.text[2:9] + '-' + prod.text[-1])
  for prod in soup.find_all('viaTransporteNome'):
    dis['Modal'] = (prod.text)
  for prod in soup.find_all('localEmbarqueTotalDolares'):
    dis['VMLE'] = float(prod.text)/100
  for prod in soup.find_all('localDescargaTotalDolares'):
    dis['VMLD'] = float(prod.text)/100
  for prod in soup.find_all('iiAliquotaValorRecolher'):
    dis['II'] = float(prod.text)/100
  for prod in soup.find_all('ipiAliquotaValorRecolher'):
    dis['IPI'] = float(prod.text)/100
  for prod in soup.find_all('pisPasepAliquotaValorRecolher'):
    dis['PIS'] = float(prod.text)/100
  for prod in soup.find_all('cofinsAliquotaValorRecolher'):
    dis['COFINS'] = float(prod.text)/100
  for prod in soup.find_all('totalAdicoes'):
    dis['adições'] = int(prod.text)
  for prod in soup.find_all('freteValorMoedaNegociada'):
    dis['Frete'] = float(prod.text)/100
  for prod in soup.find_all('freteMoedaNegociadaCodigo'):
    dis['Moeda Frete'] = (prod.text)
  for prod in soup.find_all('seguroValorMoedaNegociada'):
    dis['Seguro'] = float(prod.text)/100
  for prod in soup.find_all('seguroMoedaNegociadaCodigo'):
    dis['Moeda Seguro'] = (prod.text)
  for prod in soup.find_all('descricaoMercadoria'):
    lista = []
    lista.append(prod.text[:30])
    dis['Mercadoria'] = lista[0]
  return dis

In [204]:
def taxas(**kwargs):
  """Muitos valores necessitam ser convertidos, sejam para euros ou dolar, e essa função nos traz as taxas de acordo com a data de registro da declaração"""
  kwargs['Taxa dolar DI'] = get_exchange_rates(CurrencyCodes.USD, dt=kwargs['Data Registro'] - timedelta(1)).selling_rate
  kwargs['Taxa euro DI'] = get_exchange_rates(CurrencyCodes.EUR, dt=kwargs['Data Registro'] - timedelta(1)).selling_rate
  return kwargs

In [205]:
def taxa_siscomex(tabela_siscomex, **kwargs):
  """função que realiza o calculo da taxa siscomex, buscando a informação na tabela, usando as adições para tal"""
  Qtd_XML = int(kwargs['adições'])
  Valor_Siscomex = tabela_siscomex['Valor'][Qtd_XML-1]
  kwargs['Siscomex'] = float(Valor_Siscomex)
  return kwargs

In [206]:
def total_impostos(**kwargs):
  """calculo da soma total dos impostos da declaração"""
  kwargs['Total impostos'] = kwargs['II'] + kwargs['IPI'] + kwargs['PIS'] + kwargs['COFINS'] + kwargs['Siscomex']
  return kwargs

In [207]:
def cif(**kwargs):
  """calculo do valor cif que deve ser informado na fatura, arredondado com duas casas decimais"""
  kwargs['CIF'] = round(kwargs['VMLD'] * kwargs['Taxa dolar DI'], 2)
  return kwargs

In [208]:
def fob(**kwargs):
  """calculo do valor fob que deve ser informado na fatura, arredondado com duas casas decimais"""
  kwargs['FOB'] = round(kwargs['VMLE'] * kwargs['Taxa dolar DI'], 2)
  return kwargs

In [209]:
def frete_dolar(**kwargs):
  """o cliente necessita que o frete seja informado em dolar, então nós realizamos as conversões se necessárias e criamos um campo para o valor com essa função"""
  if kwargs['Moeda Frete'] == '790':
    kwargs['Frete_dolar'] = round(kwargs['Frete'] / kwargs['Taxa dolar DI'], 2)
  elif kwargs['Moeda Frete'] == '978':
    euro_p_real = kwargs['Frete'] * kwargs['Taxa euro DI']
    kwargs['Frete_dolar'] = round(euro_p_real / kwargs['Taxa dolar DI'], 2)
  elif kwargs['Moeda Frete'] == '220':
    kwargs['Frete_dolar'] = round(kwargs['Frete'], 2)
  else:
    kwargs['Frete_dolar'] = 0
  return kwargs

In [210]:
def seguro_dolar(**kwargs):
  """o cliente necessita que o seguro seja informado em dolar, então nós realizamos as conversões se necessárias e criamos um campo para o valor com essa função"""
  if kwargs['Moeda Seguro'] == '790':
    kwargs['Seguro_dolar'] = round(kwargs['Seguro'] / kwargs['Taxa dolar DI'], 2)
  elif kwargs['Moeda Seguro'] == '978':
    euro_p_real = kwargs['Seguro'] * kwargs['Taxa euro DI']
    kwargs['Seguro_dolar'] = round(euro_p_real / kwargs['Taxa dolar DI'], 2)
  elif kwargs['Moeda Seguro'] == '220':
    kwargs['Seguro_dolar'] = round(kwargs['Seguro'], 2)
  else:
    kwargs['Seguro_dolar'] = 0
  return kwargs

In [211]:
def abrir_arquivo(arquivo):
  """função usada para abrirmos o arquivo da Declaração de importação adequadamente para podermos extrair informações."""
  with open('drive/MyDrive/Colab Notebooks/espelho/xml/' + arquivo)as fp:
    soup = bs(fp, 'xml')
  return soup

In [212]:
def ajuste_peso(**kwargs):
  """função criada para fazermos uma pequena modificação nos campos de peso"""
  kwargs['Peso Bruto'] = str(kwargs['Peso Bruto']) + 'Kgs'
  kwargs['Peso Liquido'] = str(kwargs['Peso Liquido']) + 'Kgs'
  return kwargs

In [213]:
def excel(**kwargs):
  """função usada para carregar o modelo da fatura, e apos isso, inserir as informações  em cada campo específico, salvando com a referencia do cliente como titulo."""
  book = load_workbook('drive/MyDrive/Colab Notebooks/espelho/modele.xlsx') 
  writer = pd.ExcelWriter('drive/MyDrive/Colab Notebooks/espelho/espelho.xlsx', engine='openpyxl')
  writer.book = book
  ws = book.active
  ws['C26'] = kwargs['IV']
  ws['C27'] = kwargs['Ref. Cliente']
  ws['C32'] = kwargs['Modal']
  ws['C33'] = kwargs['Numero DI']
  ws['C34'] = kwargs['Data Registro']
  ws['C35'] = kwargs['Conhecimento']
  ws['C36'] = kwargs['Peso Bruto']
  ws['C37'] = kwargs['Peso Liquido']
  ws['C38'] = kwargs['Volume']
  ws['C39'] = kwargs['Mercadoria']
  ws['C41'] = kwargs['CIF']
  ws['C46'] = kwargs['VMLE']
  ws['C47'] = kwargs['Frete_dolar']
  ws['C48'] = kwargs['Seguro_dolar']
  ws['C49'] = kwargs['VMLD']
  ws['C53'] = kwargs['II']
  ws['C54'] = kwargs['IPI']
  ws['C55'] = kwargs['Siscomex']
  ws['C56'] = kwargs['PIS']
  ws['C57'] = kwargs['COFINS']
  ws['C58'] = kwargs['Total impostos']
  ws['I9'] = datetime.now()
  ws['H14'] = (datetime.now() + timedelta(20))
  titulo = kwargs['Ref. Cliente']
  writer.book.save(f'drive/MyDrive/Colab Notebooks/espelho/{titulo}.xlsx')
  writer.book.close()

In [214]:
for cont in range(len(arq)):
    dis = extracao(abrir_arquivo(arq[cont]))
    dis = taxa_siscomex(tabela_siscomex,**dis)
    dis = total_impostos(**dis)
    dis = taxas(**dis)
    dis = cif(**dis)
    dis = fob(**dis)
    dis = frete_dolar(**dis)
    dis = seguro_dolar(**dis)
    dis = ajuste_peso(**dis)
    excel(**dis)
